In [23]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [59]:
df = pd.read_pickle("./3_COMB_File_CLEANED_1.pkl")

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42208 entries, 0 to 42207
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   MDR_REPORT_KEY                  42208 non-null  object        
 1   MDR_TEXT_KEY                    42208 non-null  object        
 2   TEXT_TYPE_CODE                  42208 non-null  object        
 3   PATIENT_SEQUENCE_NUMBER         42208 non-null  object        
 4   DATE_REPORT                     0 non-null      object        
 5   FOI_TEXT                        42208 non-null  object        
 6   textLen                         0 non-null      float64       
 7   REPORT_NUMBER                   42208 non-null  object        
 8   REPORT_SOURCE_CODE              42208 non-null  object        
 9   MANUFACTURER_LINK_FLAG_         42208 non-null  object        
 10  ADVERSE_EVENT_FLAG              42208 non-null  object        
 11  PR

In [61]:
# create bag of words
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df.CLEAN_FOI_TEXT)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df.index
data_dtm

,aab,aai,aair,ab,abadon,abandon,abandond,abandondedcap,abandonedcap,abandonedremov,...,york,young,youth,yr,ysmal,zero,zinc,zoll,zone,zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42204,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42206,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
# checking if index is the same
# it is

In [63]:
data_dtm[data_dtm["zoo"]==1]

,aab,aai,aair,ab,abadon,abandon,abandond,abandondedcap,abandonedcap,abandonedremov,...,york,young,youth,yr,ysmal,zero,zinc,zoll,zone,zoo
17961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [64]:
data_dtm.iloc[17961]

aab       0
aai       0
aair      0
ab        0
abadon    0
         ..
zero      0
zinc      0
zoll      0
zone      0
zoo       1
Name: 17961, Length: 7424, dtype: int64

In [65]:
print(str(df.iloc[17961]["CLEAN_FOI_TEXT"]) + "\n" + str(df.iloc[17961]["MDR_REPORT_KEY"]) + "\n" + str(df.iloc[17961]["RECALL_CLASS"]))

mfgid crm receiv inform the patient daughter the patient pass away mfgid patient servic repres spoke the daughter character convers an alleg the daughter said devic lead didnt shock heart back daughter not know caus death believ wa cardiacrel patient servic repres ask daughter contact funer home request return the devic analysi patient mfgid field repres report the patient cardiologist implantingfollow physician not contact regard patient death they believ caus death noncardiac they not consultednotifi repres report had been call interrog devic that patient daughter contact hospit possibl donat devic the local zoo potenti use one their anim
1751208
0


In [66]:
# train-test split

In [67]:
X = data_dtm
y = df["RECALL_CLASS"]

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [69]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [70]:
yPredNB = classifier.predict(X_test)
yPredNB

array(['1', '1', '2', ..., '1', '0', '1'], dtype='<U1')

In [71]:
print(confusion_matrix(y_test,yPredNB))
print('\n')
print(classification_report(y_test,yPredNB))

[[2841 4562 2464]
 [ 137 1246  356]
 [ 106  135  816]]


              precision    recall  f1-score   support

           0       0.92      0.29      0.44      9867
           1       0.21      0.72      0.32      1739
           2       0.22      0.77      0.35      1057

    accuracy                           0.39     12663
   macro avg       0.45      0.59      0.37     12663
weighted avg       0.77      0.39      0.42     12663

